# Cloud Logging Tutorial: Classification of Natural Images

In this tutorial, we will guide you step-by-step through using the Obz AI python package to monitor and explain a machine learning model that classifies natural images to object labels. 

For anomaly detection, we utilize a data inspection routine based on Gaussian Mixture Model (GMM) fitted on First Order Image Features. 
For explainable AI, we compute attention maps and class-discriminative attention maps (CDAMs).

**What will you learn?**

- How to apply a simple image classification model using sample medical images.
- How to log directly to the Obz AI cloud platform from your code.

First let's install `obzai` package if you haven't already installed it. Also let's install `gdown` to download pretrained weights.

In [ ]:
%pip install obzai
%pip install -q gdown

Import the necessary modules:

In [1]:
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torch import nn
import torch
import gdown
import os

Before you get started logging and visualizing data with the `ObzClient` from our Python library, you will need your unique API key. This key is required to securely connect your code to your account on our SaaS platform.

**Here's how to find your API key:**

1. **Log in to your Dashboard:**  
   Open your web browser and go to [Obz.AI](http://app.obz.ai/). Log in using your registered email and password.

2. **Navigate to the Settings section:**  
   After logging in, look for a menu item called "Settings". Click on it to access your API settings. Make sure to copy this key now — you will need it in the next steps.

In [16]:
# Security is important. Do not share your API token with anyone.
API_TOKEN:str = "your-api-token"

Before you can monitor or visualize the workings of your model using our library, you first need to define the model you want to track.

In this tutorial, we will use a Vision Transformer (ViT) model that has been previously trained and fine-tuned on a subset of the ImageNet data called *Imagenette*.

In [3]:
def download_weights(url, output_dir, filename):
    """
    Downloads weights from the given URL if they are not already downloaded.
    """
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, filename)
    
    if not os.path.exists(output_path):
        print(f"Downloading weights to {output_path}...")
        gdown.download(url, output_path)
    else:
        print(f"Weights already exist at {output_path}. Skipping download.")

url = "https://drive.google.com/uc?id=1zV21_xkjk6YamJ6Yrj6d5u1FzxC-oY37"
output_dir = "tuned_models"
filename = "imagenette_dino_s8.pth"
download_weights(url, output_dir, filename)

Weights already exist at tuned_models/imagenette_dino_s8.pth. Skipping download.


## Configure the ViT classifer based on DINO backbone

We are adding a binary classification head (see how ```torch.nn.Linear```) onto a DINO backbone.

In [4]:
from transformers import ViTConfig, ViTModel

class DINO(nn.Module):
    """
    DINO Transformer model based on Huggingface implementation.
    """
    def __init__(self):
        super().__init__()
        # Backbone
        config = ViTConfig.from_pretrained('facebook/dino-vits8', attn_implementation="eager") # We propose eager implementation to return att scores gracefully.
        self.backbone = ViTModel(config)
        # Classfication head
        self.head = torch.nn.Linear(384, 10)
    
    def forward(self, x: torch.Tensor, output_attentions:bool=False):
        out = self.backbone(x, output_attentions=output_attentions)
        x = out["pooler_output"]
        x = self.head(x)
        if output_attentions:
            att = out["attentions"]
            return x, att
        else:
            return x

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
WEIGHTS_PATH = "./tuned_models/imagenette_dino_s8.pth"

MODEL = DINO()
MODEL.load_state_dict(torch.load(WEIGHTS_PATH, weights_only=True, map_location=torch.device(DEVICE)))
MODEL = MODEL.to(DEVICE).eval()

Next, let's prepare the datasets. In general, you will need two separate sets of images:

**Reference Dataset**: This dataset will be used to extract reference image features and to fit the outlier detection models. In ML, this may be a training dataset.

**Inference Dataset**: This dataset will be treated as incoming new data on which you want to perform outlier detection. In ML, this may be testing or validation dataset or any new samples in production.

For the tutorial case we will use ImageNet subset called: **Imagenette**. This dataset is readily available via **torchvision** package.

In [6]:
from torchvision.datasets import Imagenette

# Transforms
TRANSFORMS = v2.Compose([v2.ToImage(), 
                         v2.ToDtype(torch.float32, scale=True), 
                         v2.CenterCrop(size=(160,160)),
                         v2.Resize(size=(224,224))])
NORMALIZE = v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Datasets
train_ds = Imagenette(root="./example_data", split='train', size='160px', transform=TRANSFORMS, download=False)
ref_set = torch.utils.data.Subset(train_ds, indices=[i for i in range(0, len(train_ds), 20)])
val_ds = Imagenette(root="./example_data", split='val', size='160px', transform=TRANSFORMS)

# DataLoaders
ref_loader = DataLoader(ref_set, batch_size=32, shuffle=False)
inf_loader = DataLoader(val_ds, batch_size=6, shuffle=True)

# Labels mapping
CLASS_NAMES = ["tench", "English springer", "cassette player", 
               "chain saw", "church", "French horn", "garbage truck", 
               "gas pump", "golf ball", "parachute"]

LOGIT2NAME = {
    0: "tench",
    1: "English springer",
    2: "cassette player",
    3: "chain saw",
    4: "church",
    5: "French horn",
    6: "garbage truck",
    7: "gas pump",
    8: "golf ball",
    9: "parachute"
}

## Using ObzClient to Explain and Monitor a AI Model

In this section, we'll show you how to set up and use the `ObzClient` to automatically log your data features and model explanation outputs to the Obz AI cloud platform.

### Local Usage

You can use the **Data Inspector Module** and **XAI Module** locally on your machine. To explore all the core features of these modules, please see the Jupyter notebook tutorials provided:
- [data_inspector_tutorial.ipynb](./data_inspector_tutorial.ipynb)
- [xai_tools_tutorial.ipynb](./xai_tools_tutorial.ipynb)

### Cloud Logging with ObzClient

To take full advantage of Obz AI, you can log your data features and model explanation outputs to the Obz AI cloud. This enables you to:
- Track and organize input features and outlier statistics.
- Store and visualize model explanations and results.
- Collaborate and share insights via the Obz dashboard.

### Step 1: Instantiate the Modules

Instantiate and fit an Outlier Detector from the `Data Inspector` Module for outlier detection:

In [7]:
# Setup OutlierDetector
from obzai.data_inspector.extractor import FirstOrderExtractor
from obzai.data_inspector.detector import GMMDetector

# Choose desired feature extractor. Chosen extractor will be used for monitoring.
first_order_extrc = FirstOrderExtractor()

# Pass choosen extractor(s) to chosen OutlierDetector. Below we utilize outlier detector based on Gaussian Mixture Models.
gmm_detector = GMMDetector(extractors=[first_order_extrc], n_components=3, outlier_quantile=0.01, show_progress=True)
# Call .fit() method with passed reference dataloader. 
# Method will extract desired image features and fit outlier detection model (in that case GMM).
gmm_detector.fit(ref_loader)

Extracting features with FirstOrderExtractor: 100%|██████████| 15/15 [00:25<00:00,  1.72s/batch]


Instantiate your chosen explainability method from the `XAI Tool` Module:

In [8]:
# Setup XAI Tools
from obzai.xai.xai_tool import CDAM, AttentionMap

# Choose desired XAI Tools
cdam_tool = CDAM(model=MODEL, 
                 mode='vanilla',                      # CDAM mode
                 gradient_type="from_probabilities",  # Whether backpropagate gradients from logits or probabilities.
                 gradient_reduction="average",        # Gradient reduction method.
                 activation_type="softmax")           # Activation function applied on logits. (Needed when gradients are backpropagated from probabilities.)
# In CDAM you need to specify on which layer you want to create hooks.
cdam_tool.create_hooks(layer_name="backbone.encoder.layer.11.layernorm_before")

attention_tool = AttentionMap(model=MODEL,
                              attention_layer_id = -1,# ID of an attention layer from which to extract attention weights
                              head = None             # ID of attention head to choose. If None, attention scores are averaged.
                              )

#### Step 2: Prepare `ObzClient` for a cloud logging

Now, when you have your fitted `OutlierDetector` instance and `XAITool` instances, you are ready to wrap it into your `ObzClient`!  
It will take care of logging data to your cloud dashboard.

In [9]:
from obzai.client.obz_client import ObzClient

client = ObzClient(detector=gmm_detector, 
                     xai_tools=[cdam_tool, attention_tool],
                     verbose=True
                     )

#### Step 3: Authenticate `ObzClient` 

To use our library and send logs from your project to your online dashboard, you first need to authenticate your client. Authentication links your logs to your specific account and project, ensuring that you can securely access and manage your data.

To authenticate, you will need an API key. This key is a unique identifier for your account and allows the library to send information to your dashboard. You can find your API key by logging into your account and visiting your dashboard. If you haven't registered yet, please create an account and follow the instructions to generate your API key.

Once you have the API key, pass it to the client’s `.login()` method in your Python code.

In [11]:
# Login into client
client.login(api_key=API_TOKEN, relogin=True)

[INFO] Login successful


Entry for 'obz-backend' already exists in .netrc. Skipping.


After you log in successfully, your credentials are automatically stored in a hidden `.netrc` file on your computer. This means you **do not have to enter your API key again** when using the `ObzClient`, authentication will happen automatically in future sessions.

Now let's initialize a project with `.init_project()` method. This method accepts following arguments:  

- **`project_name`** *(str, required)*: the name of the project you want to work with.  
  - If the project already exists, you will be connected to it.
  - If it is a new name, a new project will be created automatically.

- **`ml_task`** *(str, required)*:  the type of machine learning task you are working on. We will use `multiclass_classification` in this tutorial.  
  - This setting allows ObzClient to tailor the project’s dashboard and logging features to your specific task.

- **`index2name`** *(dict, optional)*: A dictionary mapping numeric class indices to human-readable class names.  
  - Providing this helps ObzClient visualize your results with actual class names, making your dashboard easier to understand.

In [12]:
# Project initialization
client.init_project(project_name="imagenet_tutorial",
                    ml_task="multiclass_classification",
                    logit2name=LOGIT2NAME)

[INFO] Project initialized (ID=45).


#### Step 4: Log data features and model explanations to the Obz AI cloud platform

Now you can log your reference data. Simply call `.log_reference()` method on your client object. It will automatically send all reference data from your Outlier Detector into your Dashboard.

In [14]:
client.log_reference(reference_name="imagenet_tutorial_ref")

[INFO] Succesfully created ref entry.
[INFO] Uploaded chunk 1/10 successfully.
[INFO] Uploaded chunk 2/10 successfully.
[INFO] Uploaded chunk 3/10 successfully.
[INFO] Uploaded chunk 4/10 successfully.
[INFO] Uploaded chunk 5/10 successfully.
[INFO] Uploaded chunk 6/10 successfully.
[INFO] Uploaded chunk 7/10 successfully.
[INFO] Uploaded chunk 8/10 successfully.
[INFO] Uploaded chunk 9/10 successfully.
[INFO] Uploaded chunk 10/10 successfully.
[INFO] All chunks uploaded successfully.


With `ObzClient` initiated with an outlier detector and an XAI tool, we are ready to `.run_and_log`, processing batches of samples in the inference data. 

In [15]:
for idx, (image_batch, _) in enumerate(inf_loader):
    if idx > 10:
        break
    
    print("Batch: ", idx)

    # STEP 0: Take a fresh batch of images:
    image_batch, _ = next(iter(inf_loader))
    image_batch = image_batch.to(DEVICE)

    # STEP 1: Make inference:
    res = client.run_and_log(model=MODEL, image_batch=image_batch, transform=NORMALIZE)

Batch:  0


[INFO] Uploaded batch of data from 6 images.


Batch:  1


[INFO] Uploaded batch of data from 6 images.


Batch:  2


...

[INFO] Uploaded batch of data from 6 images.


Batch:  9


[INFO] Uploaded batch of data from 6 images.


Batch:  10


[INFO] Uploaded batch of data from 6 images.
